In [1]:
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
import os
from pickle import dump
from pickle import load
from sklearn import svm, metrics, datasets
from sklearn.utils import Bunch
from sklearn.model_selection import GridSearchCV, train_test_split
import skimage
from skimage.io import imread
from skimage.transform import resize
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from warnings import simplefilter

simplefilter(action='ignore', category=FutureWarning)


    Load image files with categories as subfolder names 
    which performs like scikit-learn sample dataset
    
    Parameters
    ----------
    dir_path : string  
        Path to the main folder holding one subfolder per category
    dimension : tuple
        size to which image are adjusted to (resized)
        
    Returns
    -------
    Bunch


In [2]:
def load_image_files(dir_path, dimension=(64, 64)):
    image_dir = Path(dir_path)
    folders = [directory for directory in image_dir.iterdir() if directory.is_dir()]
    categories = [fo.name for fo in folders]

    descr = "Skin Lesion - Malanoma Dataset"
    images = []
    flat_data = []
    target = []
    for i, direc in enumerate(folders):
        for file in direc.iterdir():
            img = imread(file)
            img = skimage.io.imread(file)
            img_resized = resize(img, dimension, anti_aliasing=True, mode='reflect')
            flat_data.append(img_resized.flatten()) 
            images.append(img_resized)
            target.append(i)
    flat_data = np.array(flat_data)
    target = np.array(target)
    images = np.array(images)
    return Bunch(data=flat_data,
                 target=target,
                 target_names=categories,
                 images=images,
                 DESCR=descr)

In [3]:
def load_unseen_image(filename, dimension=(64, 64)): 
    descr = "Unseen Dataset"	
    images = []
    flat_data = []
    #target = []
    img = imread(filename)
    img = skimage.io.imread(filename)
    img_resized = resize(img, dimension, anti_aliasing=True, mode='reflect')
    flat_data.append(img_resized.flatten()) 
    images.append(img_resized)
    #target.append(i)
    flat_data = np.array(flat_data)
    #target = np.array(target)
    images = np.array(images)
    return Bunch(data=flat_data, images=images, DESCR=descr) 

In [4]:
print('Loading Images...')
image_dataset = load_image_files("images/")
print('Loaded.')

Loading Images...
Loaded.


In [5]:
# Split data
print('Split Training and Testing data...')
X_train, X_test, y_train, y_test = train_test_split(image_dataset.data, image_dataset.target, test_size=0.2,random_state=109)
print('\nX_Test')
print(X_test)
print('\nX_Test[0]')
print(X_test[0])
print('\nX_Test Length')
print(len(X_test))
print('\nData[0]')
print(image_dataset.data[0])
print('\nTargrt[0]')
print(image_dataset.target[0])
print('----------------------------------------------')
print('\nTargets')
print(image_dataset.target)
print('----------------------------------------------')
print('\nTarget Length')
print(len(image_dataset.target))
print('----------------------------------------------')
print('\nTarget Names')
print(image_dataset.target_names)

Split Training and Testing data...

X_Test
[[0.87058824 0.83529412 0.81568627 ... 0.90196078 0.86666667 0.87058824]
 [0.23137255 0.11764706 0.08627451 ... 0.16470588 0.12156863 0.11372549]
 [0.20392157 0.09019608 0.09803922 ... 0.16862745 0.07058824 0.05490196]
 ...
 [0.10980392 0.10980392 0.07843137 ... 0.07058824 0.09019608 0.10196078]
 [0.10196078 0.04313725 0.03137255 ... 0.07843137 0.01176471 0.04313725]
 [0.09803922 0.05882353 0.05490196 ... 0.09019608 0.05490196 0.03529412]]

X_Test[0]
[0.87058824 0.83529412 0.81568627 ... 0.90196078 0.86666667 0.87058824]

X_Test Length
401

Data[0]
[0.56470588 0.7254902  0.89803922 ... 0.70196078 0.90196078 0.98431373]

Targrt[0]
0
----------------------------------------------

Targets
[0 0 0 ... 1 1 1]
----------------------------------------------

Target Length
2001
----------------------------------------------

Target Names
['Benign', 'Malignant']


In [6]:
# Train data with parameter optimization
param_grid = [
  {'C': [1, 10, 100, 1000], 'kernel': ['linear','poly']},
  {'C': [1, 10, 100, 1000], 'gamma': [0.001, 0.0001], 'kernel': ['poly','rbf']},
 ]

In [7]:
svc = svm.SVC()

In [8]:
clf = GridSearchCV(estimator=svc,cv = 10, param_grid=param_grid, refit=True)
print('Creating GridSearchCV Model with cv = 10') 
model = clf.fit(X_train, y_train)
print('Model created')

print('\nGridSearchCV Model')
print(clf)
# Save the model to disk
filename = 'Melanoma.pkl'
print('Saving Trained Model to ' + filename)
dump(model, open(filename, 'wb'))
print('Model Saved')

Creating GridSearchCV Model with cv = 10
Model created

GridSearchCV Model
GridSearchCV(cv=10, estimator=SVC(),
             param_grid=[{'C': [1, 10, 100, 1000],
                          'kernel': ['linear', 'poly']},
                         {'C': [1, 10, 100, 1000], 'gamma': [0.001, 0.0001],
                          'kernel': ['poly', 'rbf']}])
Saving Trained Model to Melanoma.pkl
Model Saved


In [9]:
# Load the model from disk
filename = 'Melanoma.pkl'
print('\nLoading Trained Model from Melanoma.pkl') 
loaded_model = load(open(filename, 'rb'))
result = loaded_model.score(X_test, y_test)
print('Accuracy (Loaded Model) : ', result)


Loading Trained Model from Melanoma.pkl
Accuracy (Loaded Model) :  0.8229426433915212


In [10]:
# Predict
print('Predicting...')
y_pred = loaded_model.predict(X_test)
print('Done')
print('\nPredictions')
print(y_pred)
print('\nPrediction Length : ',len(y_pred))

Predicting...
Done

Predictions
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]

Prediction Length :  401


In [11]:
# Report
print("\nClassification Report")
print(metrics.classification_report(y_test, y_pred))
print('\nY_test')
print(y_test, len(y_test))
print('\nY_Pred')
print(y_pred, len(y_pred))


Classification Report
              precision    recall  f1-score   support

           0       0.82      1.00      0.90       330
           1       0.00      0.00      0.00        71

    accuracy                           0.82       401
   macro avg       0.41      0.50      0.45       401
weighted avg       0.68      0.82      0.74       401


Y_test
[1 1 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 1 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 1 0 0 1 0 0
 0 1 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 1 0 1 0 0 0 0
 0 1 1 0 0 1 0 0 0 0 1 0 1 1 0 0 1 0 0 0 1 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0
 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0
 0 0 0 1 0 0 1 0 1 0 0 0 0 0 1 0 1 1 0 0 0 1 0 0 0 0 0 0 0 1 0 1 0 0 0 1 0
 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1
 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 1 1 0 0 0 0

C:\Users\Asus\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Asus\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Asus\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [12]:
accuracy = accuracy_score(y_test, y_pred)	
print("\nAccuracy Score: ", accuracy)


Accuracy Score:  0.8229426433915212


In [13]:
print('Total Images : ', len(image_dataset.target))
print('Total Predictions : ', len(y_pred))


Total Images :  2001
Total Predictions :  401


In [14]:
print('\nManual Prediction with X_test')
y_pred2 = loaded_model.predict(X_test[0].reshape(1, -1))
print('\nPredictions II')
print(y_pred2)

y_pred3 = loaded_model.predict(X_test[300].reshape(1, -1))
print('\nPredictions III')
print(y_pred3)

y_pred4 = loaded_model.predict(X_test[190].reshape(1, -1))
print('\nPredictions IV')
print(y_pred4)


Manual Prediction with X_test

Predictions II
[0]

Predictions III
[0]

Predictions IV
[0]


In [15]:
lst = [187,201,214,244,397]
for i in lst:
    y_predN = loaded_model.predict(X_test[i].reshape(1, -1))
    print('\nPredictions V')
    print(y_predN)


Predictions V
[0]

Predictions V
[0]

Predictions V
[0]

Predictions V
[0]

Predictions V
[0]


In [16]:
# images/Benign/ISIC_0000001.jpg
unseen = load_unseen_image('images/Benign/ISIC_0000001.jpg')
#print(len(unseen.data))
print('ISIC_0000001.jpg (Transformed) : ', unseen.data)

# Predicting loaded image (Benign)
y_predUnseen = loaded_model.predict(unseen.data.reshape(1, -1))
print('\n\nPredictions unseen loaded image')
print(y_predUnseen)

ISIC_0000001.jpg (Transformed) :  [[0.48235294 0.47843137 0.50980392 ... 0.68627451 0.67843137 0.72156863]]


Predictions unseen loaded image
[0]


In [18]:
# Predicting loaded image (Malignant)
unseen2 = load_unseen_image('images/Malignant/ISIC_0000299.jpg')
print('ISIC_00000299.jpg (Transformed) : ', unseen2.data)
y_predUnseen2 = loaded_model.predict(unseen2.data.reshape(1, -1))
print('\n\nPredictions unseen loaded image')
print(y_predUnseen2)

ISIC_00000299.jpg (Transformed) :  [[0.17647059 0.15686275 0.18431373 ... 0.1372549  0.07058824 0.10196078]]


Predictions unseen loaded image
[0]
